In [19]:
%pylab inline
import numpy as np
from scipy import special
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [20]:
METHOD_NAME = "nn_all_momentum"
DATA_FOLDER = "data/"
MODEL_FOLDER = "models/" + METHOD_NAME + "/"
OUTPUT_FOLDER = "outputs/"
TRAIN_FILE_PATH = DATA_FOLDER + "spam_train.csv"
TEST_FILE_PATH = DATA_FOLDER + "spam_test.csv"

In [21]:
if not os.path.exists(MODEL_FOLDER):
    os.makedirs(MODEL_FOLDER)
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

## Read Training Data

In [22]:
train_data = pd.read_csv(TRAIN_FILE_PATH, header=None)
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,1,0.0,0.00,1.13,0.0,0.37,0.00,0.00,0.0,0.00,...,0.0,0.145,0.000,0.436,0.000,0.0,1.792,55,147,0
1,2,0.0,0.00,0.60,0.0,0.00,0.60,0.00,0.0,0.60,...,0.0,0.143,0.047,0.191,0.143,0.0,2.041,31,196,1
2,3,0.0,0.00,0.48,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.000,0.450,0.000,0.0,1.138,4,41,0
3,4,0.0,0.51,0.00,0.0,0.51,0.51,1.02,0.0,0.00,...,0.0,0.142,0.000,0.071,1.212,0.0,7.025,130,281,1
4,5,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.64,...,0.0,0.116,0.000,0.232,0.000,0.0,1.551,6,45,0


In [23]:
x_data_rows = train_data[list(range(1, 58))].as_matrix()
y_data_rows = train_data[[58]].as_matrix()

## Select Data

## Preprocess Data

## Transform Data

In [24]:
# Normalize
x_train_means = np.mean(x_data_rows, axis=0)
x_train_stds =  np.std(x_data_rows, axis=0)
x_data_rows = (x_data_rows - x_train_means) / x_train_stds

In [25]:
# print(np.mean(x_data_rows, axis=0), np.std(x_data_rows, axis=0))

In [26]:
# add 1 in the last column, so that b in including in w
x_data_rows = np.c_[x_data_rows, np.ones(x_data_rows.shape[0]) ]

In [27]:
# pre-transpose x for matrix multiplication
x_data_rows_trans = np.transpose(x_data_rows)

In [28]:
x_row_num, x_col_num = x_data_rows.shape  # 4001, 58

## Train

In [50]:
# reset parameters
w = np.random.randn(x_col_num, 1)  # shape = (x_col_num, 1). not (1, x_col_num) for the convenience of matrix multiplication
total_epoch = 0

In [51]:
layer_1_node_num = 10

In [52]:
w_1 = np.random.randn(x_col_num, layer_1_node_num)  # dot(x, w_1) output a x_row_num * layer_1_node_num matrix
w_2 = np.random.randn(layer_1_node_num, 1)  # dot(layer_1_output, w_2) output a x_row_num * 1 matrix

In [53]:
# momentum
gamma = 0.9
vw = np.zeros((x_col_num, 1))

In [54]:
lr = (10) / x_row_num 

In [55]:
def sigmoid_deriv(x):  # input may be an matrix
    return x * (1-x)  # element-wise

In [ ]:
# training iterations

# special.expit is sigmoid

while True:
    layer_0_output = x_data_rows
    layer_1_output = special.expit(np.dot(layer_0_output, w_1))  # shape = (x_row_num, layer_1_node_num)
    layer_2_output = special.expit(np.dot(layer_1_output, w_2))  # shape = (x_row_num, 1)
    y_diff_rows = (y_data_rows - layer_2_output)  # shape = (x_row_num, 1)
    
    # backpropagation
    layer_2_error = y_diff_rows  # shape = (x_row_num, 1)
    layer_2_delta = layer_2_error * sigmoid_deriv(layer_2_output)  # shape = (x_row_num, 1)
    
    layer_1_error = layer_2_delta.dot(w_2.T)  # shape = (x_row_num, layer_1_node_num)
    layer_1_delta = layer_1_error * sigmoid_deriv(layer_1_output)  # shape = (x_row_num, layer_1_node_num)
    
    w_2 += lr * (layer_1_output.T).dot(layer_2_delta)  # shape = (layer_1_node_num, 1)
    w_1 += lr * (layer_0_output.T).dot(layer_1_delta)  # shape = (x_row_num, layer_1_node_num)
    

    # write model
    if total_epoch % 10000 == 0:
        rms_error = np.mean(np.square(y_diff_rows))
        
        model_file_info = "epo" + str(total_epoch) + "_err" + str(rms_error)[:8]
        print(model_file_info)
        print('rms_error', rms_error)
        
#         np.savez(MODEL_FOLDER + model_file_info, w=w, vw=vw, lr=lr, total_epoch=total_epoch, x_train_means=x_train_means, x_train_stds=x_train_stds)

#     vw = lr * w_grad + gamma * vw
#     w = w - vw

    total_epoch += 1
    

epo170000_err0.009944
rms_error 0.00994401691043
epo180000_err0.009920
rms_error 0.00992064538534
epo190000_err0.009899
rms_error 0.00989930961845
epo200000_err0.009879
rms_error 0.00987943275606
epo210000_err0.009860
rms_error 0.00986043451819
epo220000_err0.009842
rms_error 0.00984297343426


## Test

In [59]:
test_data = pd.read_csv(TEST_FILE_PATH, header=None)
test_data.shape

(600, 58)

In [60]:
_id = test_data[0].apply(str)

In [61]:
x_test_data_rows = test_data[list(range(1, 58))].as_matrix()

In [62]:
# read model

In [63]:
# normalize
x_test_data_rows = (x_test_data_rows - x_train_means) / x_train_stds
# add 1
x_test_data_rows = np.c_[x_test_data_rows, np.ones(x_test_data_rows.shape[0]) ]

In [64]:
def to_bool(x):
    return 1 if x > 0.5 else 0

to_bool = np.vectorize(to_bool)

In [66]:
# test
test_layer_1_output = special.expit(np.dot(x_test_data_rows, w_1))
test_layer_2_output = special.expit(np.dot(test_layer_1_output, w_2))
y_test_rows = to_bool(test_layer_2_output)

y_test_series = pd.Series(y_test_rows.flatten())

# concat id and y
output = pd.concat([_id, y_test_series], axis=1)
output.columns=["id","label"]

# write file
output.to_csv(OUTPUT_FOLDER + "output_" + METHOD_NAME + "_" + model_file_info + ".csv", index=False)